In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/MVA_Audio/

/content/drive/MyDrive/MVA_Audio


In [4]:
%ls

 Code/  'Data Set'/   Paper/   Results/


In [5]:
!pip install pyworld
!pip install pysptk

import matplotlib.pyplot as plt
import numpy as np
import os
import pyworld
import pysptk
import librosa
import time
import soundfile as sf

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

from torch.autograd import Variable
from tqdm import tqdm
from tqdm import trange
from Code.preprocess import *
from Code.model import *

torch.manual_seed(0)
np.random.seed(0)

%matplotlib inline

     |████████████████████████████████| 225kB 18.2MB/s 
  Created wheel for pyworld: filename=pyworld-0.2.12-cp37-cp37m-linux_x86_64.whl size=611747 sha256=6ce40cbf5085500a56eedf1dc9ef44e0fbf75d3efc59869f65e305fa2bf087e1
  Stored in directory: /root/.cache/pip/wheels/d0/e4/1c/a508000462b83164d5eba9a4b46f39b4b1645ac952bbe71551
Successfully built pyworld
     |████████████████████████████████| 419kB 17.9MB/s 
  Created wheel for pysptk: filename=pysptk-0.1.18-cp37-cp37m-linux_x86_64.whl size=951245 sha256=f08aa9dd985ee9ac2d90eecc49c606b35bef91dc374a77dd5b802d151ed35025
  Stored in directory: /root/.cache/pip/wheels/c7/96/d2/a163240019c59504402fab713af259026af81a99dea943404a
Successfully built pysptk


In [6]:
model_name = "best_model"
model_dir  = "Results/" + model_name

data_dir   = "Data Set/vcc2018_training/"
test_dir   = "Data Set/vcc2018_evaluation/"
data_list  = ["VCC2SF1", "VCC2SF2", "VCC2SM1", "VCC2SM2"]

test_dir_s = "Results/Sounds/"
test_dir_f = "Results/Figures/"
test_dir_t = "Results/Tests/"

lambda_p   = 70
lambda_s   = 70
nb_label   = len(data_list)

In [7]:
num_epochs = 4000
batch_size = 1
learning_rate =1e-3
learning_rate_ = 1e-4
learning_rate__ = 1e-5
learning_rate___ = 1e-6
sampling_rate = 16000
num_envelope  = 36
num_mcep = 36
frame_period = 5.0
n_frames = 1024

In [8]:
for v in data_list:
    if "log_f0_"+v+".npz" in  os.listdir(os.path.join(data_dir, v)):
        continue
    print("Preprocess: " + v)
    preprocess_voice(os.path.join(data_dir, v), v)

In [9]:
def model_save(model, model_dir, model_name):
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    torch.save(model.state_dict(), os.path.join(model_dir, model_name))
    
def model_load(model_dir, model_name):
    model = ACVAE(nb_label=nb_label,lambda_p=lambda_p,lambda_s=lambda_s)
    model.load_state_dict(torch.load(os.path.join(model_dir, model_name)))

    return model

In [10]:
def save_figure(losses, epoch):        
    if not os.path.exists(test_dir_f):
            os.makedirs(test_dir_f)
    losses = np.array(losses)
    losses = losses.reshape(-1, 4)
    x = np.linspace(0, len(losses), len(losses))
    losses_label = ("L1", "KLD", "AC_p", "AC_s")
    plt.figure()
    plt.plot(x, losses[:,0], label=losses_label[0])
    plt.plot(x, losses[:,1], label=losses_label[1])
    plt.plot(x, losses[:,2], label=losses_label[2])
    plt.plot(x, losses[:,3], label=losses_label[3])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(test_dir_f + "/" + "epoch_{:05}".format(epoch) + ".png")
    plt.savefig(test_dir_f + "/" + "result.png")
    plt.close()
        
    plt.figure()
    plt.plot(x, losses[:,2], label=losses_label[2])
    plt.plot(x, losses[:,3], label=losses_label[3])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(test_dir_f + "/" + "epoch_{:05}_AC".format(epoch) + ".png")
    plt.savefig(test_dir_f + "/" + "result_AC.png")
    plt.close()
    
    plt.figure()
    plt.plot(x, losses[:,0], label=losses_label[0])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(test_dir_f + "/" + "epoch_{:05}_L1".format(epoch) + ".png")
    plt.savefig(test_dir_f + "/" + "result_L1.png")
    plt.close()
    
    plt.figure()
    plt.plot(x, losses[:,1], label=losses_label[1])
    plt.legend(bbox_to_anchor=(1, 1), loc='upper right', borderaxespad=0)
    plt.savefig(test_dir_f + "/" + "epoch_{:05}_KLD".format(epoch) + ".png")
    plt.savefig(test_dir_f + "/" + "result_KLD.png")
    plt.close()

In [11]:
def data_load(batchsize = 1, s = -1, t = -1):
    x = []
    label = []
    for i in range(batchsize):
        if (s == -1):
            label_num = np.random.randint(nb_label)
        else:
            label_num = s
        voice_path = os.path.join(data_dir, data_list[label_num])
        files = os.listdir(voice_path)
        
        frames = 0
        #while frames < n_frames:
        file = ""
        while file.count("wav") == 0:
            file = np.random.choice(files)
        wav, _ = librosa.load(os.path.join(voice_path, file), sr = sampling_rate, mono = True)
        wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
        wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
        f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period, num_mcep = num_mcep)
        
        mc_transposed  = np.array(mc).T
        frames = np.shape(mc_transposed)[1]
            
        mcep_normalization_params = np.load(os.path.join(voice_path, "mcep_"+data_list[label_num]+".npz"))
        mcep_mean = mcep_normalization_params['mean']
        mcep_std = mcep_normalization_params['std']
        mc_norm = (mc_transposed  - mcep_mean) / mcep_std
            
        #start_ = np.random.randint(frames - n_frames + 1)
        #end_ = start_ + n_frames
            
        #x.append(mc_norm[:,start_:end_])
        n_frames = mc_norm.shape[1]
        x.append(mc_norm[:,:])
        label.append(label_num)

    return torch.Tensor(x).view(batchsize, 1, num_mcep, n_frames), torch.Tensor(label)

In [12]:
def test_conv(model, epoch):
    print("Test")
    
    if not os.path.exists(test_dir_s):
        os.makedirs(test_dir_s)
    
    output_epoch_dir = os.path.join(test_dir_s, "epoch_{:05}".format(epoch))
    if not os.path.exists(output_epoch_dir):
        os.makedirs(output_epoch_dir)
    
    for s_label in range(nb_label):
    
        voice_path_s = os.path.join(data_dir, data_list[s_label])

        files = os.listdir(voice_path_s)
        file = ""
        while file.count("wav") == 0:
            file = np.random.choice(files)
        
        print("Source File:" + file)

        for t_label in range(nb_label):

            if (t_label == s_label):
                continue

            voice_path_t = os.path.join(data_dir, data_list[t_label])

            wav, _ = librosa.load(os.path.join(voice_path_s, file), sr = sampling_rate, mono = True)
            wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
            wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
            f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period)

            mc_transposed  = np.array(mc).T

            mcep_normalization_params_s = np.load(os.path.join(voice_path_s, "mcep_"+data_list[s_label]+".npz"))
            mcep_mean_s = mcep_normalization_params_s['mean']
            mcep_std_s = mcep_normalization_params_s['std']    
            mcep_normalization_params_t = np.load(os.path.join(voice_path_t, "mcep_"+data_list[t_label]+".npz"))
            mcep_mean_t = mcep_normalization_params_t['mean']
            mcep_std_t = mcep_normalization_params_t['std']

            mc_norm = (mc_transposed - mcep_mean_s) / mcep_std_s

            x = torch.Tensor(mc_norm).view(1, 1, mc_norm.shape[0], mc_norm.shape[1])

            label_s_tensor = torch.Tensor(np.array([s_label])).view(1, 1)
            label_t_tensor = torch.Tensor(np.array([t_label])).view(1, 1)

            x = x.to(device)
            label_s_tensor = label_s_tensor.to(device)
            label_t_tensor = label_t_tensor.to(device)

            mu_enc, logvar_enc = model.encode(x, label_s_tensor)
            z_enc = model.reparameterize(mu_enc, logvar_enc)
            # x^
            mu_dec_t, logvar_dec_t = model.decode(z_enc, label_t_tensor)
            z_dec_t = model.reparameterize(mu_dec_t, logvar_dec_t)
            if (torch.cuda.is_available()):
                z_dec_t = z_dec_t.data.cpu().numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
            else:
                z_dec_t = z_dec_t.data.numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
            # x_
            mu_dec_s, logvar_dec_s = model.decode(z_enc, label_s_tensor)
            z_dec_s = model.reparameterize(mu_dec_s, logvar_dec_s)
            if (torch.cuda.is_available()):
                z_dec_s = z_dec_s.data.cpu().numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
            else:
                z_dec_s = z_dec_s.data.numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))

            mc_converted_t = z_dec_t * mcep_std_t + mcep_mean_t
            mc_converted_t = mc_converted_t.T
            mc_converted_t = np.ascontiguousarray(mc_converted_t)
            sp_converted_t = world_decode_mc(mc = mc_converted_t, fs = sampling_rate)
            mc_converted_s = z_dec_s * mcep_std_s + mcep_mean_s
            mc_converted_s = mc_converted_s.T
            mc_converted_s = np.ascontiguousarray(mc_converted_s)
            sp_converted_s = world_decode_mc(mc = mc_converted_s, fs = sampling_rate)

            sp_gained = np.multiply(sp, np.divide(sp_converted_t, sp_converted_s))

            logf0s_normalization_params_s = np.load(os.path.join(voice_path_s, "log_f0_"+data_list[s_label]+".npz"))
            logf0s_mean_s = logf0s_normalization_params_s['mean']
            logf0s_std_s = logf0s_normalization_params_s['std']
            logf0s_normalization_params_t = np.load(os.path.join(voice_path_t, "log_f0_"+data_list[t_label]+".npz"))
            logf0s_mean_t = logf0s_normalization_params_t['mean']
            logf0s_std_t = logf0s_normalization_params_t['std']

            f0_converted = pitch_conversion(f0 = f0, mean_log_src = logf0s_mean_s, std_log_src = logf0s_std_s, mean_log_target = logf0s_mean_t, std_log_target = logf0s_std_t)
            
            wav_transformed = world_speech_synthesis(f0 = f0_converted, sp = sp_gained, ap = ap, fs = sampling_rate, frame_period = frame_period)
            sf.write(os.path.join(output_epoch_dir, data_list[s_label]+"_to_"+data_list[t_label]+"_["+file+"].wav"), wav_transformed, sampling_rate)
            wav_source = world_speech_synthesis(f0 = f0_converted, sp = sp, ap = ap, fs = sampling_rate, frame_period = frame_period)
            sf.write(os.path.join(output_epoch_dir, data_list[s_label]+"_to_"+data_list[t_label]+"_["+file+"]_nonconv.wav"), wav_source, sampling_rate)

            print("Converted: " + data_list[s_label] + " -> " + data_list[t_label])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = ACVAE(nb_label=nb_label,lambda_p=lambda_p,lambda_s=lambda_s).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()

losses = []

start_time = time.time()

for epoch in range(num_epochs):
    epoch += 1
    
    if (epoch == 1000):
        learning_rate = learning_rate_ 
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 2000):
        learning_rate = learning_rate__
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 3000):
        learning_rate = learning_rate___
        for param_group in optimizer.param_groups:
            param_group['lr'] = learning_rate

    print('Epoch: %d' % epoch)

    x_, label_ = data_load(batchsize = 1)

    optimizer.zero_grad()
    loss, loss_list = model.calc_loss(x_, label_)
    loss.backward()
    losses.append(loss_list)
    optimizer.step()

    print("- Loss: {}".format(loss))
    
    if epoch % 100 == 0:
        test_conv(model, epoch)
    if epoch % 100 == 0:
        model_save(model, model_dir, model_name)
    if epoch % 2000 == 0:
        model_save(model, model_dir, model_name + "_" + str(epoch))

    if epoch % 100 == 0:
        save_figure(losses, epoch)
  
elapsed_time = time.time() - start_time
print('\nTime Elapsed: %02d:%02d:%02d' % (elapsed_time // 3600, (elapsed_time % 3600 // 60), (elapsed_time % 60 // 1)))

model_save(model, model_dir, model_name)

save_figure(losses, epoch)

cuda
Epoch: 1
- Loss: 36635.0078125
Epoch: 2
- Loss: 21007.921875
Epoch: 3
- Loss: 32933.19140625
Epoch: 4
- Loss: 19241.826171875
Epoch: 5
- Loss: 29884.587890625
Epoch: 6
- Loss: 52933.44921875
Epoch: 7
- Loss: 45264.7578125
Epoch: 8
- Loss: 19526.759765625
Epoch: 9
- Loss: 26307.1484375
Epoch: 10
- Loss: 12585.642578125
Epoch: 11
- Loss: 45252.3515625
Epoch: 12
- Loss: 30860.72265625
Epoch: 13
- Loss: 31509.47265625
Epoch: 14
- Loss: 19061.478515625
Epoch: 15
- Loss: 38722.06640625
Epoch: 16
- Loss: 36470.5078125
Epoch: 17
- Loss: 13985.1396484375
Epoch: 18
- Loss: 12945.1767578125
Epoch: 19
- Loss: 25537.734375
Epoch: 20
- Loss: 19502.064453125
Epoch: 21
- Loss: 21128.07421875
Epoch: 22
- Loss: 36796.14453125
Epoch: 23
- Loss: 50854.19140625
Epoch: 24
- Loss: 59265.06640625
Epoch: 25
- Loss: 21683.16015625
Epoch: 26
- Loss: 15685.5048828125
Epoch: 27
- Loss: 25218.439453125
Epoch: 28
- Loss: 30935.0546875
Epoch: 29
- Loss: 20758.078125
Epoch: 30
- Loss: 34072.453125
Epoch: 31
- Los

/content/drive/My Drive/MVA_Audio/Code/preprocess.py:188: RuntimeWarning: divide by zero encountered in log
  f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM1
Converted: VCC2SF1 -> VCC2SM2
Source File:10003.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM1
Converted: VCC2SF2 -> VCC2SM2
Source File:10020.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Source File:10076.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1
Epoch: 101
- Loss: 21292.626953125
Epoch: 102
- Loss: 13904.8876953125
Epoch: 103
- Loss: 18896.048828125
Epoch: 104
- Loss: 27072.208984375
Epoch: 105
- Loss: 13123.9384765625
Epoch: 106
- Loss: 26667.515625
Epoch: 107
- Loss: 24965.1328125
Epoch: 108
- Loss: 8433.326171875
Epoch: 109
- Loss: 13964.1748046875
Epoch: 110
- Loss: 18527.9140625
Epoch: 111
- Loss: 16153.064453125
Epoch: 112
- Loss: 15421.37109375
Epoch: 113
- Loss: 17819.673828125
Epoch: 114
- Loss: 14039.990234375
Epoch: 115
- Loss: 20360.75390625
Epoch: 116
- Loss: 14338.0625
Epoch: 117
- Loss: 23404.9

/content/drive/My Drive/MVA_Audio/Code/preprocess.py:188: RuntimeWarning: divide by zero encountered in log
  f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM1
Converted: VCC2SF1 -> VCC2SM2
Source File:10017.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM1
Converted: VCC2SF2 -> VCC2SM2
Source File:10037.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Source File:10055.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1
Epoch: 201
- Loss: 13259.087890625
Epoch: 202
- Loss: 33804.3359375
Epoch: 203
- Loss: 9048.880859375
Epoch: 204
- Loss: 12447.9599609375
Epoch: 205
- Loss: 16049.1064453125
Epoch: 206
- Loss: 10168.642578125
Epoch: 207
- Loss: 15635.212890625
Epoch: 208
- Loss: 15877.8603515625
Epoch: 209
- Loss: 22840.248046875
Epoch: 210
- Loss: 35096.5625
Epoch: 211
- Loss: 24980.734375
Epoch: 212
- Loss: 16212.1630859375
Epoch: 213
- Loss: 20088.646484375
Epoch: 214
- Loss: 12799.5185546875
Epoch: 215
- Loss: 13959.771484375
Epoch: 216
- Loss: 37101.8203125
Epoch: 217
- Loss: 976

...

Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM1
Converted: VCC2SF1 -> VCC2SM2
Source File:10010.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM1
Converted: VCC2SF2 -> VCC2SM2
Source File:10076.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Source File:10026.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1
Epoch: 3801
- Loss: 16674.693359375
Epoch: 3802
- Loss: 14418.1083984375
Epoch: 3803
- Loss: 34104.21875
Epoch: 3804
- Loss: 21337.310546875
Epoch: 3805
- Loss: 16685.458984375
Epoch: 3806
- Loss: 7223.2783203125
Epoch: 3807
- Loss: 16093.123046875
Epoch: 3808
- Loss: 28469.029296875
Epoch: 3809
- Loss: 11134.9287109375
Epoch: 3810
- Loss: 15779.0888671875
Epoch: 3811
- Loss: 12190.08984375
Epoch: 3812
- Loss: 9591.2861328125
Epoch: 3813
- Loss: 15232.875
Epoch: 3814
- Loss: 14212.7587890625
Epoch: 3815
- Loss: 31935.44140625
Epoch: 3816
- Loss: 9968.70703125
Epoch: 3

/content/drive/My Drive/MVA_Audio/Code/preprocess.py:188: RuntimeWarning: divide by zero encountered in log
  f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM1
Converted: VCC2SF1 -> VCC2SM2
Source File:10058.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM1
Converted: VCC2SF2 -> VCC2SM2
Source File:10069.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Source File:10054.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1
Epoch: 3901
- Loss: 11817.955078125
Epoch: 3902
- Loss: 11941.2177734375
Epoch: 3903
- Loss: 7634.77587890625
Epoch: 3904
- Loss: 31187.599609375
Epoch: 3905
- Loss: 13514.912109375
Epoch: 3906
- Loss: 20463.541015625
Epoch: 3907
- Loss: 26991.58984375
Epoch: 3908
- Loss: 13383.3427734375
Epoch: 3909
- Loss: 10126.6962890625
Epoch: 3910
- Loss: 22101.513671875
Epoch: 3911
- Loss: 23307.65625
Epoch: 3912
- Loss: 16631.287109375
Epoch: 3913
- Loss: 6980.19677734375
Epoch: 3914
- Loss: 26044.8125
Epoch: 3915
- Loss: 11249.23046875
Epoch: 3916
- Loss: 30275.609375
Epoch: 

/content/drive/My Drive/MVA_Audio/Code/preprocess.py:188: RuntimeWarning: divide by zero encountered in log
  f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)


Converted: VCC2SF1 -> VCC2SF2
Converted: VCC2SF1 -> VCC2SM1
Converted: VCC2SF1 -> VCC2SM2
Source File:10006.wav
Converted: VCC2SF2 -> VCC2SF1
Converted: VCC2SF2 -> VCC2SM1
Converted: VCC2SF2 -> VCC2SM2
Source File:10010.wav
Converted: VCC2SM1 -> VCC2SF1
Converted: VCC2SM1 -> VCC2SF2
Converted: VCC2SM1 -> VCC2SM2
Source File:10050.wav
Converted: VCC2SM2 -> VCC2SF1
Converted: VCC2SM2 -> VCC2SF2
Converted: VCC2SM2 -> VCC2SM1

Time Elapsed: 02:09:20


In [16]:
for v in data_list:
    if "log_f0_"+v+".npz" in  os.listdir(os.path.join(test_dir, v)):
        continue
    print("Preprocess: " + v)
    preprocess_voice(os.path.join(test_dir, v), v)

Preprocess: VCC2SF1
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:00:55
Preprocess: VCC2SF2
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:00:56
Preprocess: VCC2SM1
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:01:08
Preprocess: VCC2SM2
Preprocessing Data...
Data Loading...
Extracting f0 and mcep...
Saving f0 Data...
Saving mcep Data...
Preprocessing Done.
Time Elapsed for Data Preprocessing: 00:00:57


In [17]:
def conv_all(model):
    print("Conversion Start.")
    
    if not os.path.exists(test_dir_t):
        os.makedirs(test_dir_t)
    
    for s_label in range(nb_label):
        
        output_label_dir = os.path.join(test_dir_t, data_list[s_label])
        if not os.path.exists(output_label_dir):
            os.makedirs(output_label_dir)
    
        voice_path_s = os.path.join(test_dir, data_list[s_label])

        count = -1
        files = os.listdir(voice_path_s)

        for file in files:
            if file.count("wav") == 0:
                continue

            for t_label in range(nb_label):
                if (t_label == s_label):
                    continue

                voice_path_t = os.path.join(data_dir, data_list[t_label])

                wav, _ = librosa.load(os.path.join(voice_path_s, file), sr = sampling_rate, mono = True)
                wav = librosa.util.normalize(wav, norm=np.inf, axis=None)
                wav = wav_padding(wav = wav, sr = sampling_rate, frame_period = frame_period, multiple = 4)
                f0, timeaxis, sp, ap, mc = world_decompose(wav = wav, fs = sampling_rate, frame_period = frame_period)

                mc_transposed  = np.array(mc).T

                mcep_normalization_params_s = np.load(os.path.join(voice_path_s, "mcep_"+data_list[s_label]+".npz"))
                mcep_mean_s = mcep_normalization_params_s['mean']
                mcep_std_s = mcep_normalization_params_s['std']    
                mcep_normalization_params_t = np.load(os.path.join(voice_path_t, "mcep_"+data_list[t_label]+".npz"))
                mcep_mean_t = mcep_normalization_params_t['mean']
                mcep_std_t = mcep_normalization_params_t['std']

                mc_norm = (mc_transposed - mcep_mean_s) / mcep_std_s

                x = torch.Tensor(mc_norm).view(1, 1, mc_norm.shape[0], mc_norm.shape[1])

                label_s_tensor = torch.Tensor(np.array([s_label])).view(1, 1)
                label_t_tensor = torch.Tensor(np.array([t_label])).view(1, 1)

                x = x.to(device)
                label_s_tensor = label_s_tensor.to(device)
                label_t_tensor = label_t_tensor.to(device)

                mu_enc, logvar_enc = model.encode(x, label_s_tensor)
                z_enc = model.reparameterize(mu_enc, logvar_enc)
                # x^
                mu_dec_t, logvar_dec_t = model.decode(z_enc, label_t_tensor)
                z_dec_t = model.reparameterize(mu_dec_t, logvar_dec_t)
                if (torch.cuda.is_available()):
                    z_dec_t = z_dec_t.data.cpu().numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
                else:
                    z_dec_t = z_dec_t.data.numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
                # x_
                mu_dec_s, logvar_dec_s = model.decode(z_enc, label_s_tensor)
                z_dec_s = model.reparameterize(mu_dec_s, logvar_dec_s)
                if (torch.cuda.is_available()):
                    z_dec_s = z_dec_s.data.cpu().numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))
                else:
                    z_dec_s = z_dec_s.data.numpy().reshape((mc_norm.shape[0], mc_norm.shape[1]))

                mc_converted_t = z_dec_t * mcep_std_t + mcep_mean_t
                mc_converted_t = mc_converted_t.T
                mc_converted_t = np.ascontiguousarray(mc_converted_t)
                sp_converted_t = world_decode_mc(mc = mc_converted_t, fs = sampling_rate)
                mc_converted_s = z_dec_s * mcep_std_s + mcep_mean_s
                mc_converted_s = mc_converted_s.T
                mc_converted_s = np.ascontiguousarray(mc_converted_s)
                sp_converted_s = world_decode_mc(mc = mc_converted_s, fs = sampling_rate)

                sp_gained = np.multiply(sp, np.divide(sp_converted_t, sp_converted_s))

                logf0s_normalization_params_s = np.load(os.path.join(voice_path_s, "log_f0_"+data_list[s_label]+".npz"))
                logf0s_mean_s = logf0s_normalization_params_s['mean']
                logf0s_std_s = logf0s_normalization_params_s['std']
                logf0s_normalization_params_t = np.load(os.path.join(voice_path_t, "log_f0_"+data_list[t_label]+".npz"))
                logf0s_mean_t = logf0s_normalization_params_t['mean']
                logf0s_std_t = logf0s_normalization_params_t['std']

                f0_converted = pitch_conversion(f0 = f0, mean_log_src = logf0s_mean_s, std_log_src = logf0s_std_s, mean_log_target = logf0s_mean_t, std_log_target = logf0s_std_t)

                wav_transformed = world_speech_synthesis(f0 = f0_converted, sp = sp_gained, ap = ap, fs = sampling_rate, frame_period = frame_period)
                sf.write(os.path.join(output_label_dir, data_list[s_label]+"_to_"+data_list[t_label]+"_["+file+"].wav"), wav_transformed, sampling_rate)
                wav_source = world_speech_synthesis(f0 = f0_converted, sp = sp, ap = ap, fs = sampling_rate, frame_period = frame_period)
                sf.write(os.path.join(output_label_dir, data_list[s_label]+"_to_"+data_list[t_label]+"_["+file+"]_nonconv.wav"), wav_source, sampling_rate)
                
            count += 1
            if (count % 10 == 0):
                print("{} ({}/{}) : {:.1f} % is done...".format(data_list[s_label], str(s_label+1), str(nb_label), count*100/len(files)))
    print("Finish.")

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = model_load(model_dir, model_name)
model.cuda()
model.eval()
conv_all(model)

cuda
Conversion Start.


/content/drive/MyDrive/MVA_Audio/Code/preprocess.py:188: RuntimeWarning: divide by zero encountered in log
  f0_converted = np.exp((np.log(f0) - mean_log_src) / std_log_src * std_log_target + mean_log_target)


VCC2SF1 (1/4) : 0.0 % is done...
VCC2SF1 (1/4) : 26.3 % is done...
VCC2SF1 (1/4) : 52.6 % is done...
VCC2SF1 (1/4) : 78.9 % is done...
VCC2SF2 (2/4) : 0.0 % is done...
VCC2SF2 (2/4) : 27.0 % is done...
VCC2SF2 (2/4) : 54.1 % is done...
VCC2SF2 (2/4) : 81.1 % is done...
VCC2SM1 (3/4) : 0.0 % is done...
VCC2SM1 (3/4) : 27.0 % is done...
VCC2SM1 (3/4) : 54.1 % is done...
VCC2SM1 (3/4) : 81.1 % is done...
VCC2SM2 (4/4) : 0.0 % is done...
VCC2SM2 (4/4) : 27.0 % is done...
VCC2SM2 (4/4) : 54.1 % is done...
VCC2SM2 (4/4) : 81.1 % is done...
Finish.
